In [1]:
import torch
from transformers import AutoProcessor, Data2VecAudioModel
from datasets import load_dataset, Dataset, Audio, Features
from glob import glob
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
print(torch.__version__)

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1.13.1+cu117


In [2]:
annotation_list = sorted(os.listdir('/home/arplab/project/paradeigma/multi_modal/org_KEMDy20/annotation'))
file_name = annotation_list[0]
df_annotation = pd.read_csv('org_KEMDy20/annotation/' + file_name, skiprows=1)
file_lst = df_annotation[' .1']
file_lst

0      Sess01_script01_User002M_001
1      Sess01_script01_User002M_002
2      Sess01_script01_User002M_003
3      Sess01_script01_User002M_004
4      Sess01_script01_User001F_001
                   ...             
306    Sess01_script06_User001F_016
307    Sess01_script06_User002M_041
308    Sess01_script06_User002M_042
309    Sess01_script06_User002M_043
310    Sess01_script06_User001F_017
Name:  .1, Length: 311, dtype: object

## Audio Embedding

In [6]:
wav_file_lst = []
for i in file_lst:
    wav_file_lst.append('./org_KEMDy20/Session01/' + i + '.wav')
    
wav_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.wav',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.wav']

In [7]:
import datasets

dataset = Dataset.from_dict({"wav": wav_file_lst}).cast_column('wav', Audio())
dataset[0]['wav']

{'path': './org_KEMDy20/Session01/Sess01_script01_User002M_001.wav',
 'array': array([ 0.0526123 ,  0.05267334,  0.05197144, ..., -0.00387573,
        -0.00244141, -0.00137329], dtype=float32),
 'sampling_rate': 16000}

In [8]:
check_point_wav_model = 'facebook/data2vec-audio-base-960h'
processor = AutoProcessor.from_pretrained(check_point_wav_model)
model_wav = Data2VecAudioModel.from_pretrained(check_point_wav_model)

Some weights of the model checkpoint at facebook/data2vec-audio-base-960h were not used when initializing Data2VecAudioModel: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing Data2VecAudioModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecAudioModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
sampling_rate = dataset['wav'][0]['sampling_rate']
sampling_rate

16000

### audio embedding method

In [10]:
def wav_embedding_func(wav_file_lst, processor, model):
    
    padding = 'max_length'
    max_length = 16000
    truncation = True
    
    dataset = Dataset.from_dict({'wav': wav_file_lst}).cast_column('wav', Audio())
    wav_embeddings = []
    for i in range(len(dataset)):
        inputs = processor(
            dataset[i]['wav']['array'], 
            sampling_rate = sampling_rate, return_tensors = 'pt', return_attention_mask = True,
            padding = padding, max_length = max_length, truncation = truncation)
        
        with torch.no_grad():
            output = model_wav(**inputs)
            
        wav_embeddings.append(output['last_hidden_state'])
    return wav_embeddings

In [ ]:
wav_embedding_func(wav_file_lst, processor, model_wav)

## Text Embedding

In [11]:
from transformers import PreTrainedTokenizer, BartModel
from transformers import AutoTokenizer, Data2VecTextModel
from transformers import AutoModel, AutoTokenizer
import pandas as pd
import re

In [12]:
txt_file_lst = []
for i in file_lst:
    txt_file_lst.append('./org_KEMDy20/Session01/' + i + '.txt')
txt_file_lst[:3]

['./org_KEMDy20/Session01/Sess01_script01_User002M_001.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_002.txt',
 './org_KEMDy20/Session01/Sess01_script01_User002M_003.txt']

아래는 왜 다른 모델을 사용하고 있는건가요? 
의도 설명 

In [13]:
checkpoint = "facebook/data2vec-text-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model_txt = Data2VecTextModel.from_pretrained(checkpoint)
model_txt_cuda = Data2VecTextModel.from_pretrained(checkpoint).cuda()

model_txt.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at facebook/data2vec-text-base were not used when initializing Data2VecTextModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing Data2VecTextModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Data2VecTextModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Data2VecTextModel were not initialized from the model checkpoint at facebook/data2vec-text-base and are newly initialized: ['data2vec_text.pooler.dense.bias', 'data2vec_text.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it f

Embedding(50265, 768, padding_idx=1)

In [14]:
def text_embeddings_func(txt_file_lst, tokenizer, model):
    sentences = []
    padding = 'max_length'
    max_length = 80
    truncation = True
    
    for sentence in txt_file_lst:
        f = open(sentence,'r')
        line = f.readline()
        line = re.sub('\n', '', line)
        line = re.sub('  ', ' ', line)
        line = line.rstrip().lstrip()
        sentences.append(line)
        f.close()
        
    inputs = tokenizer(
        sentences, padding = padding, max_length = max_length, truncation = truncation, return_tensors='pt'
        )
    txt_embeddings, _ = model(**inputs, return_dict=False)
    return txt_embeddings
    

In [38]:
text_embeddings_func(txt_file_lst, tokenizer, model_txt)

tensor([[[ 0.1883, -0.0291, -0.0117,  ..., -0.0931, -0.0109,  0.1308],
         [-0.0740, -0.0787, -0.0656,  ..., -0.0323,  0.0825,  0.0826],
         [-0.0908,  0.1073, -0.1467,  ..., -0.0191, -0.0405,  0.0479],
         ...,
         [-0.1111,  0.1112, -0.1535,  ..., -0.0108, -0.0418,  0.0736],
         [-0.0947,  0.0485, -0.1209,  ...,  0.0095,  0.0013,  0.1210],
         [ 0.2380, -0.0621,  0.0350,  ...,  0.0772, -0.0580,  0.0579]],

        [[ 0.1796, -0.0431, -0.0118,  ..., -0.0794, -0.0236,  0.1116],
         [-0.0791, -0.0831, -0.0669,  ..., -0.0359,  0.0919,  0.0774],
         [-0.1199,  0.0889, -0.1398,  ..., -0.0478, -0.0554,  0.0616],
         ...,
         [-0.0865,  0.0340, -0.1327,  ..., -0.0668, -0.0287,  0.1769],
         [ 0.0054,  0.1212, -0.0730,  ...,  0.0018,  0.0233,  0.1023],
         [ 0.2306, -0.0696,  0.0456,  ...,  0.0123, -0.0796,  0.0533]],

        [[ 0.1986, -0.0062,  0.0027,  ..., -0.1102, -0.0632,  0.1459],
         [-0.1368, -0.0599, -0.1114,  ..., -0

In [15]:
# def build_dataset(
#     file_name_lst, 
#     text_embeddings, wav_embeddings, 
#     label_emotion, label_arousal, label_valence, 
#     session_name = 'Session01'
#     ):
    
#     dataset = {'file_names': file_name_lst, 
#          'text_embeddings': text_embeddings, 
#          'wav_embeddings':wav_embeddings,
#          'Emotion':label_emotion,
#          'Arousal':label_arousal,
#          'Valence':label_valence}
    
def build_dataset(
    file_name_lst, 
    text_embeddings, wav_embeddings,
    eda, temp ,
    label_emotion, label_arousal, label_valence, 
    session_name = 'Session01'
    ):
    
    dataset = {'file_names': file_name_lst, 
         'text_embeddings': text_embeddings, 
         'wav_embeddings':wav_embeddings,
         'EDA':eda,
         'Temp':temp,
         'Emotion':label_emotion,
         'Arousal':label_arousal,
         'Valence':label_valence
         }
    return dataset

In [16]:
session_num_lst = []
annot_num_lst = []
num_sessions = 40
for i in range(num_sessions):
    if i <= 8:
        session_num_lst.append('Session0' + str(i+1))
        annot_num_lst.append('Sess0' + str(i+1) + '_eval.csv')
        
    else:
        session_num_lst.append('Session' + str(i+1))
        annot_num_lst.append('Sess' + str(i+1) + '_eval.csv')
        
session_num_lst[-5:], annot_num_lst[-5:]

(['Session36', 'Session37', 'Session38', 'Session39', 'Session40'],
 ['Sess36_eval.csv',
  'Sess37_eval.csv',
  'Sess38_eval.csv',
  'Sess39_eval.csv',
  'Sess40_eval.csv'])

In [17]:
import pickle

with open('./model/data/paradeigma_ts_data_preprocessing.pkl', 'rb') as f:
    target_data_frames = pickle.load(f)

In [43]:
target_data_frames[1]

,segment_id,emotion,valence,arousal,eda,temp,ibi
0,Sess01_script01_User002M_001,neutral,3.4,2.9,"[-0.06791500000000017, -0.110201, -0.025627999...","[34.81, 34.81, 34.81, 34.79, 34.79, 34.79, 34....",NaN
1,Sess01_script01_User002M_002,neutral,3.1,2.9,"[0.006407000000000274, -0.021784000000000248, ...","[34.79, 34.79, 34.79, 34.77, 34.77, 34.77, 34....","[0.65625, 0.609375, 0.609375]"
2,Sess01_script01_User002M_003,neutral,3.1,3,"[0.07944700000000005, 0.05125599999999997, 0.0...","[34.75, 34.75, 34.75, 34.75, 34.79, 34.79, 34....",NaN
3,Sess01_script01_User002M_004,neutral,3.7,3.1,"[0.0038439999999999586, -0.03331699999999982, ...","[34.77, 34.77, 34.77, 34.77, 34.77, 34.77, 34....",NaN
4,Sess01_script01_User001F_001,neutral,3.8,2.8,"[0.1241910000000006, 0.09474499999999964, 0.03...","[34.55, 34.55, 34.55, 34.55, 34.55, 34.55, 34....","[0.640625, 0.640625, 0.578125]"
...,...,...,...,...,...,...,...
306,Sess01_script06_User001F_016,neutral,3.3,3.5,"[0.006400999999999435, -0.010241999999999862, ...","[34.43, 34.43, 34.43, 34.43, 34.43, 34.41, 34....",NaN
307,Sess01_script06_User002M_041,neutral,3.3,3.1,"[0.0038450000000000983, -0.007689000000000057,...","[36.05, 36.05, 36.05, 36.05, 36.03, 36.03, 36....",NaN
308,Sess01_script06_User002M_042,neutral,3,2.9,"[0.06279299999999965, 0.019222000000000072, -0...","[36.07, 36.07, 36.07, 36.07, 36.05, 36.05, 36....",NaN
309,Sess01_script06_User002M_043,neutral,3,2.7,"[-0.010251999999999928, -0.0038450000000000983...","[36.07, 36.05, 36.05, 36.05, 36.05]",NaN


In [18]:
eda_num_lst = []
temp_num_lst = []

for session in range(num_sessions):
    eda_num_lst.append(target_data_frames[session+1]['eda'])
    temp_num_lst.append(target_data_frames[session+1]['temp'])

In [45]:
eda_num_lst[:2]

[0      [-0.06791500000000017, -0.110201, -0.025627999...
 1      [0.006407000000000274, -0.021784000000000248, ...
 2      [0.07944700000000005, 0.05125599999999997, 0.0...
 3      [0.0038439999999999586, -0.03331699999999982, ...
 4      [0.1241910000000006, 0.09474499999999964, 0.03...
                              ...                        
 306    [0.006400999999999435, -0.010241999999999862, ...
 307    [0.0038450000000000983, -0.007689000000000057,...
 308    [0.06279299999999965, 0.019222000000000072, -0...
 309    [-0.010251999999999928, -0.0038450000000000983...
 310    [-0.01408300000000029, -0.025605999999999796, ...
 Name: eda, Length: 311, dtype: object,
 0      [-0.23577899999999907, -0.17427100000000095, -...
 1      [-0.0012810000000000044, 0.0, -0.0012810000000...
 2      [0.37749000000000166, 0.14736199999999933, 0.0...
 3      [0.034598000000000795, -0.006406000000000134, ...
 4      [0.00512600000000063, -0.012814000000000547, -...
                              ..

In [19]:
dataset = {}
for annot, session, eda, temp in tqdm(zip(annot_num_lst[:2], session_num_lst[:2], eda_num_lst[:2], temp_num_lst[:2])):

    df_annotation = pd.read_csv('./org_KEMDy20/annotation/' + annot, skiprows=1)
    file_name_lst = df_annotation[' .1']
    
    txt_file_lst = []
    wav_file_lst = []
    for file_name in file_name_lst:
        txt_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.txt')
        wav_file_lst.append('./org_KEMDy20/' + session + '/' + file_name + '.wav')
        
    txt_embeddings = text_embeddings_func(txt_file_lst, tokenizer, model_txt)
    wav_embeddings = wav_embedding_func(wav_file_lst, processor, model_wav)
    
    dataset[session] = build_dataset(file_name_lst, 
                txt_embeddings, 
                wav_embeddings,
                eda,
                temp,  
                df_annotation.Emotion, 
                df_annotation.Arousal, 
                df_annotation.Valence)
    
    torch.cuda.empty_cache()
    print(f'{session}번째 작업이 완료되었습니다.')

0it [00:00, ?it/s]2023-04-02 19:21:45.108869: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-02 19:21:46.430201: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/include:/usr/local/cuda-11/lib64::/usr/local/cuda/extras/CUPTI/lib64
2023-04-02 19:21:46.430298: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11/inclu

Session01번째 작업이 완료되었습니다.


2it [01:21, 40.79s/it]

Session02번째 작업이 완료되었습니다.


In [20]:
dataset.keys()

dict_keys(['Session01', 'Session02'])

In [21]:
dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'EDA', 'Temp', 'Emotion', 'Arousal', 'Valence'])

In [24]:
import pickle
# 1_3 = eda / temp 없음
# with open('./model/data/paradeigma_dataset_1_3.pkl', 'wb') as f:
#     pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

with open('./model/data/paradeigma_dataset_1_4.pkl', 'wb') as f:
    pickle.dump(dataset, f, pickle.HIGHEST_PROTOCOL)

In [22]:
with open('./model/data/paradeigma_dataset_1_3.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset['Session01'].keys()

dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'Emotion', 'Arousal', 'Valence'])

In [3]:
import pickle

with open('./model/data/paradeigma_dataset_1_4.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset['Session01'].keys()

/home/arplab/project/paradeigma/multi_modal/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['file_names', 'text_embeddings', 'wav_embeddings', 'EDA', 'Temp', 'Emotion', 'Arousal', 'Valence'])